In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [22]:
import requests
import pandas as pd

# OpenFoodFacts API
API_URL = "https://world.openfoodfacts.org/cgi/search.pl"

PAGE_SIZE = 200

params = {
    'action': 'process',
    'json': 1,
    'fields': 'product_name,nutrition_grades,ecoscore_score,energy_100g,sugars_100g,saturated-fat_100g,salt_100g,proteins_100g,fiber_100g,fruits-vegetables-nuts_100g',
    'page_size': PAGE_SIZE,
}

def fetch_data(page=1):
    params['page'] = page
    response = requests.get(API_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        return data['products']
    else:
        print("Error fetching data:", response.status_code)
        return []

def fetch_multiple_pages(total_pages=10):
    all_products = []
    for page in range(1, total_pages + 1):
        print(f"Fetching page {page}...")
        products = fetch_data(page)
        if products:
            all_products.extend(products)
        else:
            break
    return all_products

def clean_and_format_data(products):
    formatted_data = []
    
    for product in products:
        food_name = product.get('product_name', 'Unknown')
        nutri_score = product.get('nutrition_grades', 'Unknown')
        ecoscore = product.get('ecoscore_score', 'Unknown')
        energy_kj = product.get('energy_100g', 0)  # Energy (KJ)
        sugars = product.get('sugars_100g', 0)  # Sugars (g)
        saturated_fat = product.get('saturated-fat_100g', 0)  # Saturated fatty acids (g)
        salt = product.get('salt_100g', 0)  # Salt (g)
        proteins = product.get('proteins_100g', 0)  # Proteins (g)
        fiber = product.get('fiber_100g', 0)  # Fiber (g)
        fruit_veg_percentage = product.get('fruits-vegetables-nuts_100g', 0)  # Fruit/Vegetable (%)
        
        if nutri_score not in ['a', 'b', 'c', 'd', 'e']:
            continue
        
        try:
            ecoscore = float(ecoscore)  
        except ValueError:
            continue 
        
        formatted_data.append({
            'Food Name': food_name,
            'Energy (KJ/100g)': energy_kj,
            'Sugars (g/100g)': sugars,
            'Saturated Fat (g/100g)': saturated_fat,
            'Salt (g/100g)': salt,
            'Proteins (g/100g)': proteins,
            'Fiber (g/100g)': fiber,
            'Fruit/Veg (%)': fruit_veg_percentage,
            'Nutri-Score': nutri_score,
            'Ecoscore': ecoscore
        })
    
    return formatted_data

def save_to_excel(data, filename="food_data_filtered.xlsx"):
    df = pd.DataFrame(data)
    df.to_excel(filename, index=False)
    print(f"Data saved to {filename}")

def main():
    products = fetch_multiple_pages(total_pages=10)
    
    formatted_data = clean_and_format_data(products)
    
    save_to_excel(formatted_data)

if __name__ == "__main__":
    main()

Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Fetching page 6...
Fetching page 7...
Fetching page 8...
Fetching page 9...
Fetching page 10...
Data saved to food_data_filtered.xlsx


In [23]:
# clean data
df = pd.read_excel("/kaggle/working/food_data_filtered.xlsx")

df_cleaned = df.dropna(subset=['Food Name'])

df_cleaned = df_cleaned.drop_duplicates(subset=['Food Name'], keep='first')

df_cleaned.to_excel("/kaggle/working/food_data_cleaned.xlsx", index=False)

print("Data has been cleaned and saved to food_data_cleaned.xlsx")


Data has been cleaned and saved to food_data_cleaned.xlsx


In [ ]:
# check duplication rate
df = pd.read_excel("/kaggle/working/food_data_othergroup.xlsx")

duplicates = df.duplicated(subset=['Food Name'], keep=False)

duplicate_percentage = duplicates.sum() / len(df) * 100

if duplicate_percentage > 30:
    print(f"Warning: Duplicate rate is {duplicate_percentage:.2f}%")
else:
    print("Duplicate rate is acceptable.")